# PowerPoint Translation using OpenAI
This notebook demonstrates how to translate PowerPoint presentations using python-pptx and OpenAI's API.

## Import required libraries

In [1]:
import json
from pptx import Presentation
import openai
from typing import List

## Define helper function for OpenAI translation

In [2]:
def prompt_chatgpt(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
        
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=message
    )
    return response.choices[0].message.content

## Function to extract text from slides

In [3]:
def extract_text_from_slides(pptx_path: str) -> List[List[str]]:
    prs = Presentation(pptx_path)
    all_texts = []
    
    for slide in prs.slides:
        slide_texts = []
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                slide_texts.append(shape.text.strip())
        all_texts.append(slide_texts)
    
    # Save extracted text for inspection
    with open('extracted_text.json', 'w') as f:
        json.dump(all_texts, f, indent=2)
    
    return all_texts

## Function to translate texts using OpenAI

In [4]:
def translate_texts(texts: List[List[str]], target_language: str = "German") -> List[List[str]]:
    texts_json = json.dumps(texts)
    prompt = f"Translate the following text elements to {target_language}. Preserve the JSON array structure exactly. Return only the translated JSON: {texts_json}"
    
    translated_json = prompt_chatgpt(prompt)
    
    # Save translated text for inspection
    with open('translated_text.json', 'w') as f:
        f.write(translated_json)
    
    return json.loads(translated_json)

## Function to update PowerPoint with translated text

In [5]:
def update_presentation(pptx_path: str, new_texts: List[List[str]], output_path: str):
    prs = Presentation(pptx_path)
    
    for slide, slide_texts in zip(prs.slides, new_texts):
        text_index = 0
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                shape.text = slide_texts[text_index]
                text_index += 1
    
    prs.save(output_path)

## Main execution

In [6]:
# Example usage
input_pptx = "input.pptx"
output_pptx = "translated.pptx"

# Extract text
texts = extract_text_from_slides(input_pptx)

# Translate
translated_texts = translate_texts(texts)

# Sanity check
if len(texts) == len(translated_texts) and \
   all(len(orig) == len(trans) for orig, trans in zip(texts, translated_texts)):
    # Update presentation
    update_presentation(input_pptx, translated_texts, output_pptx)
    print("Translation completed successfully!")
else:
    print("Translation failed: Output format mismatch!")

PackageNotFoundError: Package not found at 'input.pptx'